In [1]:
import os 
import pathlib
from xml.etree import ElementTree

import dask
import dask.array as da
from matplotlib import pyplot as plt

import SessionTools as st 
import SessionTools.two_photon

%load_ext autoreload
%autoreload 2 

%matplotlib inline

In [8]:
basedir = pathlib.PurePath('/media/mplitt/SSD_storage/2PData/12_16_2022/EPG_GCaMP7f_CsChrimsontdT_fly1/fixed_cw_remapping-000/')
session = "fixed_cw_remapping-000"
basename_input = pathlib.Path(basedir.joinpath(session))

metadata = st.two_photon.metadata.read(basename_input,pathlib.Path('.'))

In [9]:
metadata

{'layout': {'sequences': 3280,
  'frames_per_sequence': 7,
  'samples_per_pixel': 6},
 'size': {'frames': 3280,
  'channels': 2,
  'z_planes': 7,
  'y_px': 208,
  'x_px': None,
  'x_pix': 259},
 'pixel_size': {'x': 0.330179899491773, 'y': 0.330179899491773, 'z': 5.0},
 'laser_power': {'Axon 920': 229.98046875, 'Axon 1064': 0.0, '617': 0.0},
 'frame_period': 0.013502758,
 'line_period': 6.3097e-05,
 'optical_zoom': 4.834340962,
 'pmts': {0: 839.3258666992, 1: 829.0},
 'preamp_filter': 'NoFilter',
 'preamp_offsets': {0: '-0.11', 1: '-0.06'},
 'scan_mode': 'ResonantGalvo',
 'bit_depth': 13,
 'voltage_recording': {'channels': {0: {'name': 'Input 0', 'enabled': True},
   1: {'name': 'Input 1', 'enabled': True},
   2: {'name': 'Input 2', 'enabled': True},
   3: {'name': 'Input 3', 'enabled': True},
   4: {'name': 'Input 4', 'enabled': True},
   5: {'name': 'Input 5', 'enabled': True},
   6: {'name': 'Input 6', 'enabled': True},
   7: {'name': 'Input 7', 'enabled': True}},
  'acquisition_rate

In [ ]:
basedir = pathlib.PurePath('/media/mplitt/SSD_storage/2PData/12_16_2022/EPG_GCaMP7f_CsChrimsontdT_fly1/fixed_cw_remapping-000/')
session = "fixed_cw_remapping-000"


basename_input = pathlib.Path(basedir.joinpath(session))
fname_xml = basename_input.with_suffix('.xml')
print(os.path.exists(fname_xml))
fname_vr_xml = pathlib.Path(str(basename_input) + '_Cycle00001_VoltageRecording_001').with_suffix('.xml')



fname_mp_xml = pathlib.Path(str(basename_input) + '_Cycle00001_MarkPoints').with_suffix('.xml')

mdata_root = ElementTree.parse(fname_xml).getroot()
    
class MetadataError(Exception):
    """Error while extracting metadata."""
# st.two_photon.metadata.read(basename_input,pathlib.Path('.'))

In [ ]:
def state_value(key, type_fn=str):
    element = mdata_root.find(f'.//PVStateValue[@key="{key}"]')
    value = element.attrib['value']
    return type_fn(value)

def indexed_value(key, index, type_fn=None, required=True):
    element = mdata_root.find(f'.//PVStateValue[@key="{key}"]/IndexedValue[@index="{index}"]')
    if element is None:
        if required:
            raise MetadataError('Could not find required key:index of %s:%s' % (key, index))
        return None
    value = element.attrib['value']
    return type_fn(value)

In [ ]:
sequences = mdata_root.findall('Sequence')
num_sequences = len(sequences)

num_frames_per_sequence = len(sequences[0].findall('Frame'))

if num_sequences == 1:
    num_frames = num_frames_per_sequence
    num_z_planes = 1
else:
    # If the last sequence has a different number of frames, ignore it.
    num_frames_last_sequence = len(sequences[-1].findall('Frame'))
    if num_frames_per_sequence != num_frames_last_sequence:
        logging.warning('Skipping final stack because it was found with fewer z-planes (%d, expected: %d).',
                        num_frames_last_sequence, num_frames_per_sequence)
        num_sequences -= 1
    num_frames = num_sequences
    num_z_planes = num_frames_per_sequence


In [ ]:
num_channels = len(mdata_root.find('Sequence/Frame').findall('File'))
num_y_px = state_value('linesPerFrame', int)
num_x_px = state_value('pixelsPerLine', int)

In [ ]:
laser_power = indexed_value('laserPower', '0', float, required=False)

In [ ]:
mdata_root.findall('pmtGain')

In [ ]:
element = mdata_root.find(f'.//PVStateValue[@key="preampOffset"]/SubindexedValues[@index="0"]') #/SubIndexedValue[@subindex="0"]')
subelement = element.find('.//SubindexedValue[@subindex="0"]').attrib
print(subelement)

In [ ]:
print(element)

In [ ]:
element.findall('.//SubindexedValue[@subindex="0"]')

In [ ]:
element.value

In [ ]:
laser_power

In [ ]:
voltage_root = ElementTree.parse(fname_vr_xml).getroot()

In [ ]:
channels = {}
for signal in voltage_root.findall('Experiment/SignalList/VRecSignal'):
    channel_num = int(signal.find('Channel').text)
    channel_name = signal.find('Name').text
    enabled = signal.find('Enabled').text == 'true'
    channels[channel_num] = {'name': channel_name, 'enabled': enabled}

In [ ]:
channels

In [ ]:
int(voltage_root.find('Experiment/AcquisitionTime').text)

In [ ]:
fname_vr_xml

In [ ]:
fname_mp_xml 

In [ ]:
markpoints_root = ElementTree.parse(fname_mp_xml).getroot()


In [ ]:
markpoints_root.find('PVMarkPointElement').attrib

In [ ]:
ps = [float(p)/16384.*1000 for p in markpoints_root.find('PVMarkPointElement').attrib['CustomLaserPower'].split(',')]

In [ ]:
ps

In [ ]:
element = markpoints_root.findall('PVMarkPointElement/PVGalvoPointElement/Point')

In [ ]:
element

In [ ]:
points = markpoints_root.findall('PVMarkPointElement/PVGalvoPointElement/Point')
for point in points:
    print(point.attrib)